In [37]:
import os
import sys
%pylab

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %pylab
from src import data

from src import efficient_frontier
from src import filter_assets
from src import plot
from src import portfolio
from src import capm
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
import datetime
from dateutil import rrule

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\Tor Osted\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['rrule', 'plot']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [32]:
x = datetime.datetime(2007, 6, 1)
x1 = datetime.datetime(2022,6,1)


In [33]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1400
max_esg_score = 2000
df = pd.read_excel(r"C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\data\ESG_DATA_S&P500.xlsx")
dates = [x,x1]

start_year = dates[0]
end_year = dates[1]

Bounds1 = Bounds(-1,2) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01
covariance_window = 5 #has to be in years



esg_data = data.esg_score_weight(df,weights,min_esg_score,max_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds1, sharpe_type,start_year,end_year, Wanted_return, maximum_risk,'yearly') #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters) #Gives us the weights of the individual stocks in our portfolio
portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0] #Return based on how much we allocate to the risk free asset.
sp500 = data.data_for_beta(np.append(['SPY'],(weights_of_each_portfolio.columns)),dates)
pct_returns_sp500 = data.pct_returns_from_prices(sp500)
betas_of_portfolio = capm.calculate_portfolio_beta(pct_returns_sp500,prices,weights_of_each_portfolio)
capm_expected_return = capm.capm_calc(pct_returns_sp500['SPY'].mean(),risk_free_rate,betas_of_portfolio)





c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 22.389% => Return: 20.360%  Sharpe Ratio = 0.91
Max. Sharpe Ratio = 0.99 => Return: 24.03%  Risk: 24.332%
Excpected return on investment is 101.99945471431602%


In [34]:

x = datetime.datetime(2018, 6, 1)
x1 = datetime.datetime(2022,6,1)
start = 2015
end = 2022
covariance_window = 10
listparameters = []
list_of_port_weights =[]
list_of_port_esg_scores = []
list_of_port_allocations = []
list_of_cmle_returns = []
sp500_list = []
list_of_pct_returns_sp500 = []
betas_of_portfolios = []
capm_expected_returns_list = []
for i in range(end-start):
    listparameters.append(portfolio.efficient_frontier_solo(pct_returns,
                            Bounds1,
                            sharpe_type,
                            datetime.datetime(start-covariance_window+i,1,1),
                            datetime.datetime(end-covariance_window+i,1,1),
                            Wanted_return,
                            maximum_risk)) 
    list_of_port_weights.append(efficient_frontier.weights_of_portfolio(prices,listparameters[i]))
    list_of_port_esg_scores.append(portfolio.esg_score_of_portfolio(list_of_port_weights[i],esgdata.head(1)))
    list_of_port_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
    list_of_cmle_returns.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[0])





esg_data = data.esg_score_weight(df,weights,min_esg_score,max_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
pct_returns = data.pct_returns_from_prices(prices)
esg_df = df

strategy1 = [esg_df,
             weights,
             min_esg_score,
             max_esg_score,
             Bounds1,
             sharpe_type,
             start,
             end,
             Wanted_return,
             maximum_risk,
             'monthly']



Min. Risk = 27.090% => Return: 10.855%  Sharpe Ratio = 0.40
Max. Sharpe Ratio = 0.82 => Return: 45.96%  Risk: 55.718%


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 25.346% => Return: 9.723%  Sharpe Ratio = 0.38
Max. Sharpe Ratio = 0.91 => Return: 54.55%  Risk: 60.017%


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 24.142% => Return: 14.058%  Sharpe Ratio = 0.58
Max. Sharpe Ratio = 0.91 => Return: 33.98%  Risk: 37.534%


...

Min. Risk = 22.568% => Return: 9.349%  Sharpe Ratio = 0.41
Max. Sharpe Ratio = 0.73 => Return: 29.17%  Risk: 39.843%


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 21.662% => Return: 24.489%  Sharpe Ratio = 1.13
Max. Sharpe Ratio = 1.32 => Return: 33.34%  Risk: 25.269%
Min. Risk = 20.645% => Return: 18.811%  Sharpe Ratio = 0.91
Max. Sharpe Ratio = 1.06 => Return: 25.70%  Risk: 24.134%


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 18.985% => Return: 23.610%  Sharpe Ratio = 1.24
Max. Sharpe Ratio = 1.34 => Return: 27.39%  Risk: 20.453%


In [38]:
#use numpy datetime, works much better apparently
def backtesting(strategy,monthly_or_yearly_rebalancing,rebalancing_freq,start_date,end_date,covariance_window):
    #start = 2015
   # end = 2022

    covariance_window_time_delta = datetime.timedelta(weeks=52.1429*covariance_window) #The time delta for the covariance window
    esg_data = data.esg_score_weight(strategy[0],strategy[1],strategy[2],strategy[3])
    full_data = data.stock_monthly_close(esg_data,[start_date-covariance_window_time_delta,end_date])
    prices,esgdata = data.seperate_full_data(full_data)
    pct_returns = data.pct_returns_from_prices(prices)
    
    
    listparameters = []
    list_of_port_weights =[]
    list_of_port_esg_scores = []
    list_of_port_allocations = []
    list_of_cmle_returns = []
    sp500_list = []
    list_of_pct_returns_sp500 = []
    betas_of_portfolios = []
    i = 0
    if monthly_or_yearly_rebalancing == 'yearly':
        delta = datetime.timedelta(days=rebalancing_freq*365)
        
        for dt in rrule.rrule(rrule.YEARLY*rebalancing_freq, dtstart = start_date, until=end_date):
            print(dt,start_date)
            

            listparameters.append(portfolio.efficient_frontier_solo(pct_returns,
                            Bounds1,
                            sharpe_type,
                            start_date+np.timedelta64(covariance_window,'Y'),#The start date for portfolio optimization will be the start date minus the covariance window
                            dt,#The end date for portfolio optimization will be the start date minus the covariance window
                            Wanted_return,
                            maximum_risk,
                            strategy[10])) 
            list_of_port_weights.append(efficient_frontier.weights_of_portfolio(prices,listparameters[i]))
            list_of_port_esg_scores.append(portfolio.esg_score_of_portfolio(list_of_port_weights[i],esgdata.head(1)))
            list_of_port_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            list_of_cmle_returns.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[0])
            

        return(list_of_port_weights,list_of_port_esg_scores,list_of_port_allocations,list_of_cmle_returns)

    elif monthly_or_yearly_rebalancing == 'monthly':
        delta = datetime.timedelta(weeks =rebalancing_freq*4)
        while (start_date <= end_date):#The covariance window is given in years

            listparameters.append(portfolio.efficient_frontier_solo(pct_returns,
                            Bounds1,
                            sharpe_type,
                            start_date-covariance_window_time_delta,#The start date for portfolio optimization will be the start date minus the covariance window
                            start_date,#The end date for portfolio optimization will be the start date minus the covariance window
                            Wanted_return,
                            maximum_risk,
                            strategy[10])) 
            list_of_port_weights.append(efficient_frontier.weights_of_portfolio(prices,listparameters[i]))
            list_of_port_esg_scores.append(portfolio.esg_score_of_portfolio(list_of_port_weights[i],esgdata.head(1)))
            list_of_port_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            list_of_cmle_returns.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[0])
            start_date += delta
            end_date += delta
            i += 1
        return(list_of_port_weights,list_of_port_esg_scores,list_of_port_allocations,list_of_cmle_returns)

    
    else:
        print('You can only call this function with monthly or yearly')
        return(False)
    

backtesting(strategy1,'yearly',1,datetime.date(2015,1,1),datetime.date(2022,1,1),10) 

strategy1 = [esg_df,
             weights,
             min_esg_score,
             max_esg_score,
             Bounds1,
             sharpe_type,
             start,
             end,
             Wanted_return,
             maximum_risk,
             'monthly']

#This makes each portfolio using strategy 1 rebalancing every 6 months from 2015 until 2022, using a covariance window of ten years

#This makes each portfolio for 


2015-01-01 00:00:00 2015-01-01


UFuncTypeError: ufunc 'add' cannot use operands with types dtype('O') and dtype('<m8[Y]')

In [ ]:
#5.807% ,5.712%

portfolio.efficient_frontier_solo(pct_returns,Bounds1, sharpe_type, datetime.date(2015,1,1),datetime.date(2022,1,1), Wanted_return,maximum_risk,'monthly')

c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:44: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:117: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 6.141% => Return: 2.562%  Sharpe Ratio = 0.42
Max. Sharpe Ratio = 0.43 => Return: 2.66%  Risk: 6.263%


[(0.06262768058568673,
  0.026639824108082973,
  0.061409859302649354,
  0.02561532361999815,
  array([0.06142662, 0.0614253 , 0.06142397, 0.06142264, 0.06142129,
         0.06141995, 0.06141861, 0.0614173 , 0.06141602, 0.06141478,
         0.06141361, 0.06141253, 0.06141157, 0.06141078, 0.06141019,
         0.06140988, 0.06140993, 0.06141043, 0.06141153, 0.0614134 ,
         0.06141627, 0.06142046, 0.06142637, 0.06143459, 0.06144588,
         0.06146135, 0.06148256, 0.06151177, 0.06155234, 0.06160937,
         0.06169079, 0.06180937, 0.10270563, 0.10270563, 0.10270563,
         0.10270563, 0.10270563, 0.10270563, 0.10270563, 0.10270563,
         0.10270563, 0.10270563, 0.06141109, 0.11158095, 0.13953574,
         0.13953574, 0.13953574, 0.13953574, 0.13953574, 0.13953574]),
  array([-0.3       , -0.29224654, -0.28449308, -0.27673962, -0.26898617,
         -0.26123271, -0.25347925, -0.24572579, -0.23797233, -0.23021887,
         -0.22246541, -0.21471196, -0.2069585 , -0.19920504, -0.19